# Aesthetic Feature Extraction

##### By Aesthetic feature extraction we refer to the aesthetical components of an image this mainly refers to Color, Composition and Texture of an adcreative. This features are extracted on the top of the assumption the peoples are attracted to beauty as a whole eventhough different persons have different perception of beauty.

In [ ]:
class ImageColor():
    def __init__(self, image_path):
        super(ImageColor, self).__init__()

        # Read in the image
        self.bgr_img = None
        self.hsv_img = None
        self.gray_img = None

        self.h_mean, self.s_mean, self.v_mean = None, None, None
        self.h_std, self.s_std, self.v_std = None, None, None

        # Read in the image
        self.read_in(image_path)

    def update(self, image_path):
        self.read_in(image_path)